In [ ]:
import os
import numpy as np
import cv2

path = 'C:/Users/jumpr_000/Desktop/ransac_obstacle_detection/saved'
frames_dir = 'kinect_data/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.uint8(np.load(file_path))
        cv2.imwrite(os.path.join(path ,filename[:-4] + '.png'), img)
cv2.destroyAllWindows()

In [ ]:
import os
import numpy as np
import cv2
import time
import math
import pandas as pd
from obstacle_detection import get_obstacles_with_plane
from depth_image_processing import *
import csv
import matplotlib.pyplot as plt
from scipy.optimize import minimize

path = 'kinect_data/'

num_planes = np.arange(2,32,1)#[2,5,8,16, 32]
num_points = np.arange(3,32,1)#[3,5,10,16, 32]
dist_thresh = [0.1, 0.01, 0.05, 0.2]

def get_performance(x):
    planes = int(x[0])
    points = int(x[1])
    thresh = int(x[2])
    file_reader= open('test.csv', "rt", encoding='ascii')
    read = list(csv.reader(file_reader))
    
    accuracies = np.array([])
    
    skip = False
    
    for i in range(0,len(read)):
        if skip:
            skip = False
            continue
        row = read[i]
        img_path = path + row[0][:-4] + '.npy'
        if row[0] == 'img':
            continue
        frame = np.load(img_path)
        label = np.zeros(frame.shape, dtype = np.uint8)
        label = cv2.rectangle(label, (int(row[3]), int(row[5])), (int(row[4]), int(row[6])),(255,255,255), -1)
        if i < len(read) - 1:
            if row[0] == read[i + 1][0]:
                next_row = read[i + 1]
                label = cv2.rectangle(label, (int(next_row[3]), int(next_row[5])), (int(next_row[4]), int(next_row[6])),(255,255,255), -1)
                skip = True
        
        
        
        obstacles = get_obstacles_with_plane(frame, num_planes = planes, 
                                      num_points = points, 
                                      dist_thresh = thresh, 
                                      visualize = False)
        img = np.uint8(obstacles)
        intersect = cv2.bitwise_and(img, label) #find correct detetions
        accuracy = cv2.countNonZero(intersect) #count number of pixels correctly detected
        incorrect = cv2.countNonZero(cv2.subtract(img,label)) #count pixels outside of label
        truth = cv2.countNonZero(label)
        #print('Accuracy:', accuracy, 'Incorrect:', incorrect)
        accuracies = np.append(accuracies, incorrect / truth)
        key = cv2.waitKey(delay=1)
    return np.mean(accuracies)



#x0 = [10, 5, 0.1]
#res = minimize(get_performance, x0, method='nelder-mead',
#               options={'xtol': 1e-8, 'disp': True}) 

performance_graph = np.array([])
planes_graph = np.array([])
points_graph = np.array([])
thresh_graph = np.array([])
steps = np.array([])
step = 0
min_params = [10,5,.1]
loss_graph = np.array([])
min_loss = 1.0
for thresh in dist_thresh:
    for point in num_points:
       for plane in num_planes:
            steps = np.append(steps, step)
            points_graph = np.append(points_graph, point)
            planes_graph = np.append(planes_graph, plane)
            thresh_graph = np.append(thresh_graph, thresh)
            performance = get_performance([plane, point, thresh])
            if math.isnan(performance):
                performance = 1.0
            if performance < min_loss:
                print('New min:',performance)
                min_loss = performance
                loss_graph = np.append(loss_graph, min_loss)
                min_params = [plane, point, thresh]
            print(plane, point, thresh, performance)
            performance_graph = np.append(performance_graph, performance)
            step += 1

            
rows = 1
cols = 2
y_lists = {'loss':performance_graph,
           'min_loss':loss_graph}
graph_labels = list(y_lists.keys())
fig, axes = plt.subplots(rows, cols, figsize=(20,6))
fig.patch.set_facecolor('white')
for i in range(1,3):
    sub = plt.subplot(rows, cols, i)
    plt.plot(y_lists[graph_labels[i-1]])
    sub.set_title(graph_labels[i-1])
plt.tight_layout()
plt.show()

print('Optimal params:',min_params)


New min: 0.14796569531070886
2 3 0.1 0.14796569531070886
3 3 0.1 0.2867496782027756
4 3 0.1 0.5048028943391035
5 3 0.1 0.29013618904935484
6 3 0.1 0.228167817499409
7 3 0.1 0.23036394924329853
8 3 0.1 0.22760481142084393
9 3 0.1 0.3195128022438623
10 3 0.1 0.366119290147448
New min: 0.13535357934222397
11 3 0.1 0.13535357934222397
12 3 0.1 0.19989086554679822


C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr_000\Desktop\ransac_obstacle_detection\obstacle_detection.py:110: RuntimeWarning: invalid value encountered in less
  if mean_val < HIGH_DISTANCE_BOUND: #kinect loses accuracy beyond 4.5m


13 3 0.1 0.23543693656325934
14 3 0.1 0.149444805226455
New min: 0.12463938417623899
15 3 0.1 0.12463938417623899
16 3 0.1 0.1602237740963097
17 3 0.1 0.29114246729089865
18 3 0.1 0.14894977831936376
19 3 0.1 0.1658296927521057
20 3 0.1 0.24612290766162204
21 3 0.1 0.22603527763810707
22 3 0.1 0.19387135379501455
23 3 0.1 0.1459355530595338
24 3 0.1 0.3575930047283519
25 3 0.1 0.15792711272408702
26 3 0.1 0.3520321391098804
27 3 0.1 0.22293841828358893
28 3 0.1 0.1647161969475624
29 3 0.1 0.1691679673090973
30 3 0.1 0.22011167536726123
31 3 0.1 0.13308402022966803
New min: 0.0730527705875597
2 4 0.1 0.0730527705875597
3 4 0.1 0.28484653623428996
New min: 0.06655008262551473
4 4 0.1 0.06655008262551473
5 4 0.1 0.07318760014493265
6 4 0.1 0.19005072661765077
7 4 0.1 0.2705357021618013
8 4 0.1 0.14342804655073632
9 4 0.1 0.11000552867424114
10 4 0.1 0.2835070464291741
11 4 0.1 0.09992766555425255
12 4 0.1 0.23996730851942802
13 4 0.1 0.25791095243661283
14 4 0.1 0.11405962812764432
15 4 0

19 12 0.1 0.10221904461682643
20 12 0.1 0.07529338973636399
21 12 0.1 0.047422161400777776
22 12 0.1 0.17834131754843005
23 12 0.1 0.062002691484568884
24 12 0.1 0.11127218436171875
25 12 0.1 0.06882500762738018
26 12 0.1 0.07922590393806968
27 12 0.1 0.10504941788848161
28 12 0.1 0.26258228361765673
29 12 0.1 0.20688162640194535
30 12 0.1 0.22884363904704053
31 12 0.1 0.12129814867654909
2 13 0.1 0.0688385565095495
3 13 0.1 0.062066594937892926
4 13 0.1 0.13546434405819857
5 13 0.1 0.1048610208758393
6 13 0.1 0.04808537121569239
7 13 0.1 0.08706546042594732
8 13 0.1 0.1421987985322345
9 13 0.1 0.16350282156073778
10 13 0.1 0.1546027826206516
11 13 0.1 0.14462679909631923
12 13 0.1 0.1262665687512478
13 13 0.1 0.18518010970977802
14 13 0.1 0.1150228718298549
15 13 0.1 0.08144011017699823
16 13 0.1 0.10258091255970789
17 13 0.1 0.08249184583634417
18 13 0.1 0.09137565113576883
19 13 0.1 0.09666383002088605
20 13 0.1 0.04837624011030433
21 13 0.1 0.16027932786810914
22 13 0.1 0.104300974

24 21 0.1 0.03822212824900163
25 21 0.1 0.04203429908884056
26 21 0.1 0.0224687392414265
27 21 0.1 0.10698458772791551
28 21 0.1 0.02511823482166472
29 21 0.1 0.05917507407151533
30 21 0.1 0.025917314255653825
31 21 0.1 0.06815964947523791
2 22 0.1 0.04813988847414879
3 22 0.1 0.1019341559523379
4 22 0.1 0.077589068997488
5 22 0.1 0.08229430584212707
New min: 0.009831872356184665
6 22 0.1 0.009831872356184665
7 22 0.1 0.06805918182262335
8 22 0.1 0.06556053414063948
9 22 0.1 0.06371056429070905
10 22 0.1 0.06551969447422888
11 22 0.1 0.03280598097501679
12 22 0.1 0.08498946597424979
13 22 0.1 0.07137620083780684
14 22 0.1 0.05743020490187735
15 22 0.1 0.10854285761074081
16 22 0.1 0.08342090357592237
17 22 0.1 0.08583746796910438
18 22 0.1 0.11455860142817978
19 22 0.1 0.06099205183617683
20 22 0.1 0.11125564213492702
21 22 0.1 0.028051968615565782
22 22 0.1 0.22700974571413943
23 22 0.1 0.06875406513324508
24 22 0.1 0.04597632703632133
25 22 0.1 0.04148967404326352
26 22 0.1 0.1838460

30 30 0.1 0.0710993030921875
31 30 0.1 0.02661646201451623
2 31 0.1 0.10727611641792248
3 31 0.1 0.08460131473869817
4 31 0.1 0.14670214566409026
5 31 0.1 0.07566023506193983
6 31 0.1 0.07966624167769733
7 31 0.1 0.18794172384431926
8 31 0.1 0.08448107959899574
9 31 0.1 0.02976037290633396
10 31 0.1 0.028792497162533705
11 31 0.1 0.03677860090563607
12 31 0.1 0.10322614915933269
13 31 0.1 0.06904792769853006
14 31 0.1 0.043990747551512575
15 31 0.1 0.0610069060757087
16 31 0.1 0.0955971100625851
17 31 0.1 0.04434922110036922
18 31 0.1 0.09227554967901722
19 31 0.1 0.04746133778822583
20 31 0.1 0.11211613383339447
21 31 0.1 0.11572466035791766
22 31 0.1 0.025261961526443565
23 31 0.1 0.05701391894407696
24 31 0.1 0.041427017290303576
25 31 0.1 0.11731287713061225
26 31 0.1 0.056846429644458206
27 31 0.1 0.04025545511049663
28 31 0.1 0.05403391836121437
29 31 0.1 0.07643342045335917
30 31 0.1 0.11264000820135908
31 31 0.1 0.03359393696921854
2 3 0.01 0.16960407119074455
3 3 0.01 0.098196

7 11 0.01 0.13810383606210005
8 11 0.01 0.24872888170583526
9 11 0.01 0.14419649653271885
10 11 0.01 0.06405231561122215
11 11 0.01 0.10114442583461027
12 11 0.01 0.24307438305400436
13 11 0.01 0.1186616208235193
14 11 0.01 0.12343980974037892
15 11 0.01 0.08269711234438216
16 11 0.01 0.14272866649564686
17 11 0.01 0.03005475748084635
18 11 0.01 0.19902437562427838
19 11 0.01 0.12254777053570999
20 11 0.01 0.1675042361798264
21 11 0.01 0.0832285347754526
22 11 0.01 0.05265321836934827
23 11 0.01 0.2026584499191547
24 11 0.01 0.08607308609308588
25 11 0.01 0.06666160085517023
26 11 0.01 0.14853400915188283
27 11 0.01 0.16106579471527296
28 11 0.01 0.18461576491896353
29 11 0.01 0.09361871712304944
30 11 0.01 0.18052345088905505
31 11 0.01 0.10071588716177986
2 12 0.01 0.19077510081257595
3 12 0.01 0.1417812421843643
4 12 0.01 0.11463966314642968
5 12 0.01 0.1299771639860621
6 12 0.01 0.11954738639330835
7 12 0.01 0.12178354835518779
8 12 0.01 0.10842272385845297
9 12 0.01 0.193785638188

In [ ]:
plt.plot(performance_graph)
plt.plot(planes_graph)
plt.plot(points_graph)
plt.plot(thresh_graph)
plt.show()